## MATH 404 Computational Assignment
## Quadratic Sieve
## Samia Zaman, Teodora Petkova

To run this project, click on run all cells, from the "Cell" drop-down meny and enter a $n$ of your choice in the prompt below. Then scroll down to the last cell to see the prime factors of $n$ and the time it took the code to find them. <br>

In [29]:
import math
import numpy as np
from sympy import *
import time 

#### Choose the value of N:

In [30]:
start = time.time()
n = int(input())

# Set the bound according to n
k = math.log(n)
k = k*math.log(k)
k = math.exp(math.sqrt(k))
k = math.pow(k, math.sqrt(2)/4)
bound = int(k)


8876645503455770083


#### Function block:

This cell contains all of the functions used in our code. Each function has  its inputs, outputs, and a short description of what it does in the comments above it. 

In [31]:
# input: start and end time stamps
# output: the times which passed between the time stamps
# The function is used to measure the time for our algorithm.

def calc_time(start, end):
    seconds = end - start
    mins = seconds / 60
    print('Time in seconds: ', str(round(seconds, 4)), ', Time in minutes: ', str(round(mins, 4)))
    return seconds, mins

### ----------------------------------------------

# input: a positive integer bound
# output: a list of all primes less than or equal to bound
# The function is an implementation of Eratosthenes' sieve.

def get_primes_less_thanB(bound):                      # eratosthenes way
    check_until = int(bound**0.5)
    unmarked = np.array([x for x in range(2, bound)])
    q = 2
    i = 0
    while True:
        u = unmarked[i]
        if u> check_until:               # only check numbers up to sqrt(bound)
            break
        u_idx = np.where(unmarked == u)[0][0]
        max_idx = bound//u
        listof_multiples = [u*i for i in range(q, max_idx+1)]
        u_multiples = np.array(listof_multiples)
        unmarked = np.setdiff1d(unmarked, u_multiples)   # numpy set diff without changing order of elements in list
        q = u
        i +=1
    print("Number of elements less than bound = ", bound, " is: ", len(unmarked))
    return (unmarked)

### ----------------------------------------------

# input: a list of primes and an integer
# output: a list of all powers of the primes from the list in the prime
#         factorization of the number. If the number contains primes 
#         larger than the primes in the list, the function returns an empty list.
# This function is used in the sieveing process, to check if the sieving candidates
# are Bsmooth or not.
def check_Bsmooth(primes, num):
    powers_of_primes= []
    num_remaining= num                         
    for prime in primes: 
        i = 0                                   # i = power of prime
        if (num_remaining%prime == 0):
            while (num_remaining%prime ==0):   #keep dividing for each power of the prime
                quotient = num_remaining/prime
                num_remaining = quotient
                i +=1
        powers_of_primes.append(i)
        if num_remaining == 1:     
            diff = len(primes)-len(powers_of_primes)
            if diff>0:
                zeros = [0 for k in range(diff)]
                return powers_of_primes + zeros
            return powers_of_primes
    return []

### ----------------------------------------------

# input: a list of primes, a number n, and a multiple of n
# output: a list containing the powers of primes for each of the Bsmooth residues,
#         a list containing the residues themselves,
#         the count of numbers checked for a Bsmooth residue,
#         the squares from which the residues mod n were derived,
#         the numbers themselves, which were squared to get the squares in the previous line

# This is the sieving function. It finds the B-smooth numbers needed for the matrix.
# It finds 10 more Bsmooth numbers than the number of primes,
# so we can always have a solution of the matrix.

def generate_BSsquares(primes, n, multiple):
    exponents_list = []             # Add other fancy ways of finding candidates for xsquare
    xsquares = []           # the x
    x_vals = []
    residues = []           # the y
    iters = 0
    while(True):
        x = int(math.sqrt(n*multiple)) +1
        xsquare = x**2
        residue = xsquare%n
        exp_vec = check_Bsmooth(primes, residue)
        if exp_vec != []:
            residues.append(residue)   # x^2
            xsquares.append(xsquare)
            exponents_list.append(exp_vec)
            x_vals.append(x)
            print("Found B-smooth square! with exponents: ", exp_vec, " and quadratic residue: ", residue)
        if len(residues) > len(primes)+10 :
            print("Breaking, len of residues: ", len(residues), " len of primes: ", len(primes))
            break
        multiple += 1
        iters += 1
    return [exponents_list, residues, iters, xsquares, x_vals]


### ----------------------------------------------

# input: a 2D matrix
# output: the input matrix without all of its rows which have only zeros and an integer evens
# The function is used later to remove all zero rows to ease the row reduction process

def drop_zero_rows(A):     
    evens = 0
    return [A[~np.all(A == 0, axis=1)], evens]

### ----------------------------------------------

# input: a vector of zeros and ones and a vector of numbers of the same size
# ouput: the product the numbers in the second vector,
#        which have a corresponding 1 in the first vector.
# A helper function for later calculations.

def get_product(v, num_list):  # just multiply all RELEVANT (depends on v) numbers in a list together
    product= 1       
    for i in range(len(v)):
        if v[i] ==0:
            continue
        product = product * num_list[i]
    return product

### ----------------------------------------------

# input: a row reduced matrix mod 2. (It is not augmented, even though the name says so.)
# output: a combination vector(described in the binary_add_arr function later) set to [0, 0, ...., 0, 1],
#         a vector with the indices of all columns which do not have a pivot value
# The function is used to initialize the combinations of non-pivot value residues which
# we want to use to find a solution of the matrix.

def set_v_comb_array(aug_matrix):
   
    # first set v to have all 0s for pivot columns, 1 for the others
    col_num = len(aug_matrix[0])
    row_num = len(aug_matrix)
    v = np.ones(col_num, dtype = 'int')
    
    for i in range(0, row_num):
        check_main = 1
        for j in range(i, col_num):
            if(aug_matrix[i][j]==1 and check_main ==1):
                v[j]=0
                break
    
    # generate v_combs
    count_free = np.sum(v); # gives us the count of the non-pivot columns.
    v_combs = np.zeros(count_free, dtype = 'int')
    v_combs[count_free-1] = 1;
    indices_of_free_vars = np.zeros(count_free, dtype= 'int')
    
    # generate indices_of_free_vars
    k=0;
    for i in range(0, col_num):
        if(v[i]==1):
            indices_of_free_vars[k] = i;
            k+=1;
    
    return [v_combs, indices_of_free_vars]

### ----------------------------------------------

#input: the row reduced matrix, the current combination vector,
#       the indices array from the function above, rows and columns numbers of the matrix.
#output: the vector used to calculate x, y. It has 0s for the residues that are not used, 1s for the residues that are used.
# This function is used to calculate the values of the pivot column residues which correspond to the current 
# combination of non-pivot column residues.

def calc_v(L, v_combs, v_indices, rows, cols):
    # create r, such that the only 1s are the free variables specified by v_combs
    # the rest of the free variables are 2s, and the non-free ones are 0s
    
    r = np.zeros(cols, dtype ='int')
    for i in range(0, len(v_combs)):
        if(v_combs[i]==1):
            r[v_indices[i]] = 1;
        else:
            r[v_indices[i]] =2;  
    
    for i in reversed(range(0, rows)):
        sum =0
        for j in range(0, cols):
            sum = sum + r[j]*L[i][j];
        zer =(np.argwhere(r == 0).flatten())
        sum = sum%2;
        if(len(zer)!=0):
            index = zer[len(zer)-1]
    
            if(sum==0):
                r[index] = 2
            else:
                r[index] = 1

    #print("final r vector is ", r%2)
    return r%2

### ----------------------------------------------

# This function is used to do generate all combinations of an input vector when called 2^{size(vector)} times.
# It works on the following principle:
# Each index of the array from which we want to generate combinations is either represented or not in the combination
# Hence, the indices of the array can be represented in an array of 0s and 1s

# Example:
# Array: [2, 3, 4, 5, 6]
# comb_arr: [0, 0, 0, 0, 1]
# combination: [6]

# comb_arr: [0, 0, 0, 1, 1]
# combination: [5, 6]

# comb_arr: [1, 0, 1, 1, 0]
# combination: [2, 4, 5]

# Hence, by starting from comb_arr = [0,0,0,...,0, 1] and incrementing the array similarly to a binary number, we get 
# an exhaustive list of all possible combinations of k elements.

def binary_add_arr(comb_arr):
    #for i in reverse(range(0, len(comb_arr))):
    i = len(comb_arr) -1
    while(comb_arr[i]==1):
        comb_arr[i] = 0;
        i-=1;
    if(i>=0):
        comb_arr[i] = 1;
    else:
        return -1
    return comb_arr

# input: two integers x, n
# output: the nth root of x
# The credit for this function goes to https://riptutorial.com/python/example/8751/computing-large-integer-roots
# It calculates accuretely roots of large nums.

def nth_root(x, n):
    # Start with some reasonable bounds around the nth root.
    upper_bound = 1
    while upper_bound ** n <= x:
        upper_bound *= 2
    lower_bound = upper_bound // 2
    # Keep searching for a better result as long as the bounds make sense.
    while lower_bound < upper_bound:
        mid = (lower_bound + upper_bound) // 2
        mid_nth = mid ** n
        if lower_bound < mid and mid_nth < x:
            lower_bound = mid
        elif upper_bound > mid and mid_nth > x:
            upper_bound = mid
        else:
            # Found perfect nth root.
            return mid
    return mid + 1

#### B-smooth numbers generation:

In [32]:
primes = np.array(get_primes_less_thanB(bound))

Number of elements less than bound =  93  is:  24


In [33]:
multiple = 1
results = generate_BSsquares(primes, n, multiple)

print("The needed Bsmooth numbers were found in this many iterations: ", results[2])
print("\nB smooth numbers found: ", results[4])

Found B-smooth square! with exponents:  [0, 3, 2, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]  and quadratic residue:  295859925
Found B-smooth square! with exponents:  [0, 2, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1, 0, 0]  and quadratic residue:  1020798462159
Found B-smooth square! with exponents:  [2, 3, 2, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]  and quadratic residue:  1183439700
Found B-smooth square! with exponents:  [1, 2, 0, 0, 0, 1, 0, 2, 0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]  and quadratic residue:  186344828982
Found B-smooth square! with exponents:  [2, 2, 2, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0]  and quadratic residue:  1755805608900
Found B-smooth square! with exponents:  [0, 5, 2, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]  and quadratic residue:  2662739325
Found B-smooth square! with exponents:  [2, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0

#### Matrix equation part of the algorithm:
The prime powers of the residues are converted to a matrix, and a vector $\mathbf v$ is built, which when multiplied by the matrix gives $\mathbf 0$. Two cases are considered - when all Bsmooth residues are squares $\pmod n$, and when there is at least one which is not a square. <br>

Each column in the matrix representes the vector of prime powers of a Bsmooth residue. Each row represents the powers of a prime less than the bound in each of the B-smooth residues.<br>

The *results* array on top provides us with:<br>
    $\qquad\cdot$ A 2D array of the powers of the primes in the corresponding residues.<br>
    $\qquad\cdot$ A vector of the Bsmooth residues $\pmod n$.<br>
    $\qquad\cdot$ The number of iterations needed to find the residues.<br>
    $\qquad\cdot$ The squares which get those residues. <br>
    $\qquad\cdot$ The roots of the squares which get those residues. <br>

In [34]:
A = np.array(results[0])      # results = [exponents_list, residues, iters, xsquares]
store_pows = A.copy()
A = (np.transpose(A))%2
print('Initial matrix mod 2:\n', A)

Initial matrix mod 2:
 [[0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0]
 [1 0 1 0 0 1 0 0 1 1 0 0 1 1 0 1 0 0 0 1 0 1 1 1 0 1 1 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 1]
 [0 1 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1]
 [0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [1 0 1 0 0 1 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 1 0]
 [1 0 1 0 0 1 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0]
 [1 1 1 0 1 1 1 1 0 1 0 0 0 1 1 1 0 1 0 0 1 0 0 1 1 0 1 0 0 1 0 1 0 1 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 1 0 1 

In [35]:
col_num = len(A[0])
row_num = len(A)
L = Matrix(A)
L = np.array(L.rref()[0])%2

print('Row reduced matrix: \n', L)# reduced row echelon form

L2  = drop_zero_rows(L)[0]        # get rid of primes we don't have to worry about in general
L2 = L2 %2    

Row reduced matrix: 
 [[1 0 1 0 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0

#### All residues are squares:

In the next cell we explore the possibility that all B-smooth residues are squares, thus after the dropping of the zero rows, the result is an empty matrix. Since any vector could be the solution to such a matrix equation, to find a solution, we build a combination vector for all residues, and multiply different combinations of them, until we get a combination of $x, y$ values for which $x\not\equiv \pm y\pmod n$.<br>

In [36]:
L2_empty =0
if(len(L2)==0):
    L2_empty = 1
    even_comb = np.zeros(col_num)
    even_comb[col_num-1] =1;
    
    residues = results[1]    
    ysq = get_product(even_comb, residues) 
    y = nth_root(ysq, 2)

    x_vals = results[4]
    x = get_product(even_comb, x_vals)
    
    while ((x-y)%n==0 or (x+y)%n==0 or(y-x)%n==0):
        even_comb = binary_add_arr(even_comb)
        x = get_product(even_comb, x_vals)
        ysq = get_product(even_comb, residues) 
        y = nth_root(ysq, 2)
        
    diff = abs(x-y)

    gcd = math.gcd(diff, n)
    
    # The factors are generated here, but printed in the bottom of the code.
    factor1 = gcd
    factor2 = int(n/gcd)
    
    end = time.time()    
    


#### There is at least one non-square residue
In the following cells we follow the same algorithm we used for the previous case, however this time we have pivot values in the matrix which need to be calculated corresponding to the values of the non-pivot indices. Hence this time instead of combinations of all residues, we consider different combinations of the non-pivot residues and their corresponding pivot residues.<br>

In [37]:
if(L2_empty==0):
    print("Row reduced echelon form mod 2 is: ")
    print(L2)


Row reduced echelon form mod 2 is: 
[[1 0 1 0 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0

In [38]:
if(L2_empty==0):
    col_num = len(L2[0])
    row_num = len(L2)
    # initialize the combination array for the non-pivot values, and the corresponding indices array
    v_combs, v_indices = set_v_comb_array(L2)
    
    #calculate the inital solution vector
    v = calc_v(L2, v_combs, v_indices, row_num, col_num)

In [39]:
if(L2_empty==0):
    residues = results[1]    # x%n, RHS
    xsquares = results[3]
    ysq = get_product(v, residues) 
    xsq = get_product(v, xsquares)
    print("final x squared mod n: ", xsq%n, " final y squared mod n: ", ysq%n) # just a check - they have to be equal


final x squared mod n:  5845538997557715397  final y squared mod n:  5845538997557715397


In [ ]:
if(L2_empty==0):
    x_vals = results[4]
    x = get_product(v, x_vals)
    y = nth_root(ysq, 2)

    # here we check if the x and y are good, and until they become good we change the combination of free variables
    while ((x-y)%n==0 or (x+y)%n==0 or(y-x)%n==0):
        v_combs = binary_add_arr(v_combs)
        v = calc_v(L2, v_combs, v_indices, row_num, col_num)
        x = get_product(v, x_vals)
        ysq = get_product(v, residues) 
        y = nth_root(ysq, 2)
    
    print("\n")
    print("Found x not congruent y mod n! A Factor!!")
    print('x is ', x)
    print('y is ' , y)

In [ ]:
if(L2_empty==0):
    diff = abs(x-y)
    gcd = math.gcd(diff, n)
    factor1 = gcd
    factor2 = int(n/gcd)

    end = time.time()

calc_time(start, end);
print('The first factor is ', factor1)
print('The second factor is ', factor2)

### THE END